In [9]:
# try to make energy expectation more efficient using cirq.PauliSum
from xmps.Hamiltonians import Hamiltonian
import cirq

ham_dict = {'ZZ':0.2, 'X':3, 'Z':1}
tfim = Hamiltonian(ham_dict)
print(tfim.strings)

letter_to_cirq_ops = {
    'X':cirq.ops.X,
    'Z':cirq.ops.Z,
    'I':cirq.ops.I
}

def to_pauli_string(qubits, ham_dict):
    L = len(qubits) 
    pauli_strings = []
    for i in range(L):
        for strings in ham_dict.keys():

            coeff = float(ham_dict[strings]) / L

            if len(strings) == 2:
                if i < (L-1):
                    pauli_strings.append(
                        coeff * cirq.PauliString(
                            letter_to_cirq_ops[strings[0]].on(qubits[i]),
                            letter_to_cirq_ops[strings[1]].on(qubits[i+1]),
                    ))

            else:
                pauli_strings.append(
                    coeff * cirq.PauliString(
                        letter_to_cirq_ops[strings].on(qubits[i])
                    )
                )

    return sum(pauli_strings)
L=4
qubits = cirq.GridQubit.rect(1,L)
x = to_pauli_string(qubits, {'ZZ':0.2, 'X':3, 'Z':1})
print(x)

{'ZZ': 0.2, 'IX': 3, 'XI': 3, 'IZ': 1, 'ZI': 1}
0.050*Z((0, 0))*Z((0, 1))+0.750*X((0, 0))+0.250*Z((0, 0))+0.050*Z((0, 1))*Z((0, 2))+0.750*X((0, 1))+0.250*Z((0, 1))+0.050*Z((0, 2))*Z((0, 3))+0.750*X((0, 2))+0.250*Z((0, 2))+0.750*X((0, 3))+0.250*Z((0, 3))


In [10]:
from mps_gradient import (
    energy, 
    energy_circuit,
    generate_params
)
import numpy as np

L = 4

p = generate_params(L)
circuit = energy_circuit(
    mps = None,
    L = L,
    p_forward=p,
    p_reverse=p,
    use_mps=False
)

assert np.allclose( np.eye(2**L), np.round(cirq.unitary(circuit)))

In [11]:
print(energy(tfim, circuit))

1.1499995


In [12]:
sim = cirq.Simulator()
psi = sim.simulate(circuit).final_state_vector
qs = cirq.GridQubit.rect(1,L)
qubit_mapping = {q:i for i,q in zip(range(L), qs)}
a = x.expectation_from_state_vector(psi, qubit_mapping).real
print(a)

1.1500000962400332


In [6]:
a / energy(tfim,circuit)

1.0000005137453367

In [15]:
from mps_gradient import energy_efficient

energy_efficient({'ZZ':0.2, 'X':3, 'Z':1}, circuit)

0.050*Z((0, 0))*Z((0, 1))+1.500*X((0, 1))+0.750*X((0, 0))+0.500*Z((0, 1))+0.250*Z((0, 0))+0.050*Z((0, 1))*Z((0, 2))+1.500*X((0, 2))+0.500*Z((0, 2))+0.050*Z((0, 2))*Z((0, 3))+0.750*X((0, 3))+0.250*Z((0, 3))


1.6499988262463223

In [1]:
%load_ext autoreload
%autoreload 2
